In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan.model import DATGAN
#from modules.datgan_preLSTM.model import DATGAN

#from modules.datgan_no_att.model import DATGAN
import networkx as nx

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/DATGAN_tmp2/'

In [10]:
datgan = DATGAN(continuous_columns, max_epoch=100, #steps_per_epoch=2000, 
                batch_size=500, output=output_folder, gpu=0)

In [11]:
test = datgan.fit(df, graph)

[1012 13:59:03 @model.py:173] Preprocessing the data!
[1012 13:59:03 @data.py:433] Encoding continuous variable "age"...
[1012 13:59:03 @data.py:292]   Found 6 peaks!
[1012 13:59:03 @data.py:300]   Encoding with 6 components.
[1012 14:00:03 @data.py:319]   Predictions were done on 6 components => FINISHED!
[1012 14:00:03 @data.py:448] Encoding categorical variable "gender"...
[1012 14:00:03 @data.py:448] Encoding categorical variable "travel_dow"...
[1012 14:00:03 @data.py:448] Encoding categorical variable "license"...


D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 10 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[1012 14:00:03 @data.py:448] Encoding categorical variable "education_level"...
[1012 14:00:03 @data.py:448] Encoding categorical variable "work_status"...
[1012 14:00:03 @data.py:448] Encoding categorical variable "hh_income"...
[1012 14:00:03 @data.py:448] Encoding categorical variable "trip_purpose"...
[1012 14:00:03 @data.py:448] Encoding categorical variable "hh_descr"...
[1012 14:00:03 @data.py:448] Encoding categorical variable "hh_size"...
[1012 14:00:04 @data.py:433] Encoding continuous variable "departure_time"...
[1012 14:00:04 @data.py:292]   Found 8 peaks!
[1012 14:00:04 @data.py:300]   Encoding with 8 components.
[1012 14:01:23 @data.py:319]   Predictions were done on 8 components => FINISHED!
[1012 14:01:23 @data.py:433] Encoding continuous variable "distance"...
[1012 14:01:23 @data.py:292]   Found 0 peaks!
[1012 14:01:23 @data.py:300]   Encoding with 10 components.


D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 10 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[1012 14:03:01 @data.py:319]   Predictions were done on 10 components => FINISHED!
[1012 14:03:02 @data.py:448] Encoding categorical variable "hh_vehicles"...
[1012 14:03:02 @data.py:448] Encoding categorical variable "hh_bikes"...
[1012 14:03:02 @data.py:448] Encoding categorical variable "choice"...


...


100%|################################################################################|175/175[00:05<00:00,32.03it/s]

[1012 14:12:25 @base.py:285] Epoch 99 (global_step 17325) finished, time:5.46 seconds.
[1012 14:12:25 @saver.py:79] Model saved to ../output/Chicago/DATGAN_tmp2/model\model-17325.
[1012 14:12:25 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.938
[1012 14:12:25 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.08
[1012 14:12:25 @monitor.py:467] GAN_loss/discrim/loss: 0.64967
[1012 14:12:25 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0182
[1012 14:12:25 @monitor.py:467] GAN_loss/gen/klloss: 0.075142
[1012 14:12:25 @monitor.py:467] GAN_loss/gen/loss: 0.94307
[1012 14:12:25 @monitor.py:467] QueueInput/queue_size: 50
[1012 14:12:25 @base.py:275] Start Epoch 100 ...



100%|################################################################################|175/175[00:05<00:00,31.98it/s]

[1012 14:12:31 @base.py:285] Epoch 100 (global_step 17500) finished, time:5.47 seconds.
[1012 14:12:31 @saver.py:79] Model saved to ../output/Chicago/DATGAN_tmp2/model\model-17500.
[1012 14:12:31 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.942
[1012 14:12:31 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.064
[1012 14:12:31 @monitor.py:467] GAN_loss/discrim/loss: 0.64507
[1012 14:12:31 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0039
[1012 14:12:31 @monitor.py:467] GAN_loss/gen/klloss: 0.053155
[1012 14:12:31 @monitor.py:467] GAN_loss/gen/loss: 0.9507
[1012 14:12:31 @monitor.py:467] QueueInput/queue_size: 50
[1012 14:12:31 @base.py:289] Training has finished!

[1012 14:12:31 @graph.py:282] Creating cell for age (in-edges: 0)


[1012 14:12:31 @graph.py:282] Creating cell for gender (in-edges: 0)
[1012 14:12:31 @graph.py:282] Creating cell for travel_dow (in-edges: 0)
[1012 14:12:31 @graph.py:282] Creating cell for license (in-edges: 1)
[1012 14:12:31 @graph.py:282] Creating cell for education_level (in-edges: 1)
[1012 14:12:31 @graph.py:282] Creating cell for work_status (in-edges: 2)
[1012 14:12:31 @graph.py:282] Creating cell for hh_income (in-edges: 2)
[1012 14:12:31 @graph.py:282] Creating cell for trip_purpose (in-edges: 1)
[1012 14:12:31 @graph.py:282] Creating cell for hh_descr (in-edges: 2)
[1012 14:12:31 @graph.py:282] Creating cell for hh_size (in-edges: 3)
[1012 14:12:31 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.
[1012 14:12:31 @graph.py:282] Creating cell for departure_time (in-edges: 1)
[1012 14:12:31 @graph.py:282] Creating cell for distance (in-edges: 1)
[1012 14:12:32 @graph.py:282] Creating cell for hh_vehicles (in-edges: 2)
[1012 14:12:32 @graph.py:282] Creating cell 

In [12]:
datgan.save('trained', force=True)

[1012 14:12:41 @model.py:355] Model saved successfully.
